# Exploratory data analysis (EDA)

Exploratory data analysis, or **EDA**, means just that. We are going look at and summarize our idea. The data cleaning that we've already done, the looking for missings and understanding our variables, is actually part of EDA. In these notes, however, we'll focus on **grouping**, or **aggregating**, and summarizing our data.

This type of data work is sometimes called **split-apply-combine**, where we split, or **group** our data, apply a function (e.g. mean), and then combine the summary statistics back together.

We are starting to **ask questions** of our data. For example,

- What variables do we have and what are their data types? Are they numeric or text?
- Which variables are **continuous**? Which are **discrete**?
- Are any variables **categorical**? These variables might be either numeric or text. 
- Do we have any **datetime** variables?
- How are our variables related to each other?
- How can we **summarize** them?

We'll graphically summarize our data in the [next section](#dataviz). We'll look more at **datetime** variables when we get to our work on [financial time series](#financial-times-series). 

We want to eventually be able to open up a new data, see what questions **can be answered**, and then have the tools to actually **answer the questions**. Sometimes we want to ask questions that data can't answer - we need to recognize when that's the case too.

[DataCamp](https://www.datacamp.com/community/tutorials/exploratory-data-analysis-python), of course, has a tutorial on some EDA ideas, perhaps more related to what people are doing in machine learning.

[Chapter 10](https://wesmckinney.com/book/data-aggregation.html) of *Python for Data Analysis* discusses aggregation and summarizing our data. 

[Exploratory Data Analysis](https://aeturrell.github.io/coding-for-economists/data-exploratory-analysis.html) from *Coding for Economists* has data aggregation and summary examples. That chapter also covers the logic of opening up a data set and seeing what you have to work with. I am following along with a lot of that material in these notes.


## Set-up

Let's bring in some data on NC breweries. We'll again use my Github URL to download the data directly: <https://raw.githubusercontent.com/aaiken1/fin-data-analysis-python/main/data/ncbreweries.csv>. 

We'll bring in our usual libraries. I'll also use `pyjanitor`, in case we need to clean anything.

I'll name the DataFrame `nc`. 

In [18]:
# Set-up
import numpy as np
import pandas as pd
import janitor

nc = pd.read_csv('https://raw.githubusercontent.com/aaiken1/fin-data-analysis-python/main/data/ncbreweries.csv')
nc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        251 non-null    object
 1   City        251 non-null    object
 2   Type        251 non-null    object
 3   Beer Count  251 non-null    int64 
 4   Est         251 non-null    int64 
 5   Status      251 non-null    object
 6   URL         251 non-null    object
dtypes: int64(2), object(5)
memory usage: 13.9+ KB


Looks pretty good. I'll use `pyjanitor` to clean up the variable names and get rid of any spaces and capitalizations. We're transforming them into something called **snake case**, which gets rid of capital letters and uses "_" as spaces.

In [19]:
nc = nc.clean_names()
nc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        251 non-null    object
 1   city        251 non-null    object
 2   type        251 non-null    object
 3   beer_count  251 non-null    int64 
 4   est         251 non-null    int64 
 5   status      251 non-null    object
 6   url         251 non-null    object
dtypes: int64(2), object(5)
memory usage: 13.9+ KB


See what it did? Let's take a peak at the first few rows.

In [20]:
nc.head()

,name,city,type,beer_count,est,status,url
0,217 Brew Works,Wilson,Microbrewery,10,2017,Active,https://www.ratebeer.com//brewers/217-brew-wor...
1,3rd Rock Brewing Company,Trenton,Microbrewery,12,2016,Active,https://www.ratebeer.com//brewers/3rd-rock-bre...
2,7 Clans Brewing,Cherokee,Client Brewer,1,2018,Active,https://www.ratebeer.com//brewers/7-clans-brew...
3,Andrews Brewing Company,Andrews,Microbrewery,18,2014,Active,https://www.ratebeer.com//brewers/andrews-brew...
4,Angry Troll Brewing,Elkin,Microbrewery,8,2017,Active,https://www.ratebeer.com//brewers/angry-troll-...



We can also use `pandas` to change the data types. For example, let's get brewery *type* to be something called a **category**. I'll also change a few others.

In [21]:
nc = nc.assign(
    name=nc["name"].astype('string'),
    city=nc["city"].astype('string'),
    type=nc["type"].astype('category'),
    status=nc["status"].astype('category'),
    url=nc["url"].astype('string'),
)
nc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   name        251 non-null    string  
 1   city        251 non-null    string  
 2   type        251 non-null    category
 3   beer_count  251 non-null    int64   
 4   est         251 non-null    int64   
 5   status      251 non-null    category
 6   url         251 non-null    string  
dtypes: category(2), int64(2), string(3)
memory usage: 10.7 KB


Finally, let's just check for missing data. Doesn't seem like we have any from that `info()`.

In [22]:
nc.isna().sum()

name          0
city          0
type          0
beer_count    0
est           0
status        0
url           0
dtype: int64

## Exploring NC breweries

We'll start with `describe()`. The will give us summary statistics for our numeric variables.

In [23]:
nc.describe()

,beer_count,est
count,251.000000,251.000000
mean,32.960159,2012.155378
std,43.723385,8.749158
min,1.000000,1900.000000
25%,10.000000,2011.000000
50%,18.000000,2014.000000
75%,38.000000,2016.000000
max,424.000000,2018.000000


Anything stand out to you? The variable *est* is the year that the brewery was established. The minimum is 1900. That seems very early? Is that right? A beer count of 424 seems high, but could be right.

We can make that table look a little better.

In [24]:
sum_table = nc.describe().round(1)
sum_table

,beer_count,est
count,251.0,251.0
mean,33.0,2012.2
std,43.7,8.7
min,1.0,1900.0
25%,10.0,2011.0
50%,18.0,2014.0
75%,38.0,2016.0
max,424.0,2018.0


We can flip things around, or **transpose**, the table to make it look better. This is usually how summary statistics are presented.

In [25]:
sum_table = sum_table.T
sum_table

,count,mean,std,min,25%,50%,75%,max
beer_count,251.0,33.0,43.7,1.0,10.0,18.0,38.0,424.0
est,251.0,2012.2,8.7,1900.0,2011.0,2014.0,2016.0,2018.0


Let's look at the variable *type*. What types of breweries are in our data?

In [33]:
nc['type'].value_counts()

Microbrewery          165
Brewpub/Brewery        41
Brewpub                33
Client Brewer           9
Commercial Brewery      3
Name: type, dtype: int64

Mainly microbreweries. I wonder what type of brewery has the largest number of beers on average? This is where we can start using the idea of **split-apply-combine**.

In [29]:
nc.groupby('type')['beer_count'].mean()

type
Brewpub               44.939394
Brewpub/Brewery       37.707317
Client Brewer          5.000000
Commercial Brewery    10.666667
Microbrewery          31.315152
Name: beer_count, dtype: float64

This code is **grouping** our data by *type*, pulling out *beer_count*, and calculating the mean.

We can use the `.agg` method to include more than one summary statistic. The `np.` means that the function is being pulled from the `numpy` package. Pay attention to the syntax - there are no `()` after the `np.mean`, `np.median`, and `np.std` functions now, because they are being used inside of the `.agg` function. They are also grouped together with `[]`.

The `.agg` function is the same thing as `.aggregate`, which you'll see in our textbook.

You could also pull multiple columns to summarize, but this data set really only has one numeric variable of interest. 

In [37]:
nc.groupby('type')['beer_count'].agg([np.mean, np.median, np.std]).round(1) 

,mean,median,std
type,,,
Brewpub,44.9,21.0,77.3
Brewpub/Brewery,37.7,28.0,36.3
Client Brewer,5.0,2.0,7.0
Commercial Brewery,10.7,9.0,6.7
Microbrewery,31.3,18.0,36.6


See how I added that `round(1)` to get only one decimal place. You can also make your table and then style it in separate lines. This code will style **just that table**. You can also set global style options that will affect all of your results.

In [48]:
sum_table_group = nc.groupby('type')['beer_count'].agg([np.mean, np.median, np.std])

sum_table_group.style.format(precision=0, na_rep='-')

,mean,median,std
type,,,
Brewpub,45,21,77
Brewpub/Brewery,38,28,36
Client Brewer,5,2,7
Commercial Brewery,11,9,7
Microbrewery,31,18,37


The code above reads from left to right. You do something, have a `.`, do something else, etc. For example, we are taking the DataFrame `nc`, grouping it by *type*, pulling out *beer_count*, and then aggregating using three different statistics. I am saving this to a **new DataFrame**, called `sum_table_group`. Finally, I am using `style.format` to change the look at the DataFrame that is being displayed as a table in out notebook.

Let's look at how to do this using some different syntax. Something more vertical. Note that `()` surrounding all of the code.

In [74]:
(
    nc.groupby('type')
    .mean()['beer_count']
    .to_frame()
    .style
        .format(precision=0)
        .set_caption("Beer Count by Brewery Type")   
)





,beer_count
type,
Brewpub,45
Brewpub/Brewery,38
Client Brewer,5
Commercial Brewery,11
Microbrewery,31


We are starting with the `nc` DataFrame and grouping by `type`. I am calculating the `mean` of *beer_count*. This actually creates a **series**, not a DataFrame. You can't style a series and make it look good in your output. So, I use `to.frame()` to put it back into a DataFrame that can be styled. I then use different style functions on that DataFrame. Both `format` and `set_caption` are functions found as part of `style`. 

Here's another example where I group by two variables.

In [60]:
(
    nc.groupby(['type', 'status'])
    .mean()["beer_count"]
)

type                status
Brewpub             Active    49.206897
                    Closed    14.000000
Brewpub/Brewery     Active    40.405405
                    Closed    12.750000
Client Brewer       Active     2.857143
                    Closed    12.500000
Commercial Brewery  Active    10.666667
                    Closed          NaN
Microbrewery        Active    33.173333
                    Closed    12.733333
Name: beer_count, dtype: float64

See how that doesn't look as good? We need to make it a DataFrame and style it.

In [76]:
(
    nc.groupby(['type', 'status'])
    .mean()["beer_count"]
    .to_frame()
    .fillna('-')
    .style
        .format(precision=0)
        .set_caption("Beer Count by Brewery Type and Status")
)


 You could have written this code like this too. The first line groups, creates the means, and puts all of this back into a DataFrame. The second line formats the DataFrame.

 The summary DataFrame **is our table**. 

In [83]:
sum_table_mean = nc.groupby(['type', 'status'])['beer_count'].mean().to_frame()

sum_table_mean.fillna('-').style.format(precision=0).set_caption("Beer Count by Brewery Type and Status")


### GroupBy objects

We've been splitting our data into groups. Let's look at `GroupBy` objects themselves. These are created by the `groupby` function. Our textbook discusses these, starting pg. 130. Essentially, it's another way of doing what we've already done in these notes.

Let's create a `GroupBy` object with the brewery data and take an look.

In [87]:
nc_groups = nc.groupby('type')
print(type(nc_groups))
print(nc_groups.size())

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
type
Brewpub                33
Brewpub/Brewery        41
Client Brewer           9
Commercial Brewery      3
Microbrewery          165
dtype: int64


This `GroupBy` object has all of our data, but grouped by *type*. We can calculate the mean of all numeric data in the `GroupBy`. 

In [88]:
nc_groups.mean()

,beer_count,est
type,,
Brewpub,44.939394,2010.333333
Brewpub/Brewery,37.707317,2013.487805
Client Brewer,5.000000,2009.555556
Commercial Brewery,10.666667,2016.000000
Microbrewery,31.315152,2012.260606


We can also use `.agg` again to get multiple statistics. I'll put out just *beer_count* and get the sum, mean, min, and max by group *type*. So, very similar to above, but now we are dealing with a DataFrame that's "pre-grouped" in a sense. Also, note again how I use `np.mean`, but just `sum`. The `sum` function is base Python, but `mean` comes from `numpy`, so I need the `np.mean`. 

In [90]:
nc_groups['beer_count'].agg([sum, np.mean, min, max])

,sum,mean,min,max
type,,,,
Brewpub,1483,44.939394,1,424
Brewpub/Brewery,1546,37.707317,3,165
Client Brewer,45,5.000000,1,23
Commercial Brewery,32,10.666667,5,18
Microbrewery,5167,31.315152,1,279


## skimpy

Adding in some new libraries really lets us see why using something like Python can improve our workflow. The `skimpy` library gives us a nice summary of our data. We can install `skimpy` via Anaconda. To use this package, you'll need to type the following in the terminal (Mac) or command prompt (Windows). You'll need to restart VS Code.

```
python -m pip install skimpy
```

This *should* install skimpy in your Anaconda distribution of Python. Remember, you actually have several versions of Python on your computer. This is why you have that choice of Python kernel when you're running code.

 I'm going to bring back the Zillow `uw` DataFrame, just because it has more variables to look at. 

You can read about this package [here](https://aeturrell.github.io/skimpy/). 

In [3]:
import pandas as pd
from skimpy import skim

uw = pd.read_csv('https://github.com/aaiken1/fin-data-analysis-python/raw/main/data/zestimatesAndCutoffs_byGeo_uw_2017-10-10_forDataPage.csv')

skim(uw)

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 2610   │ │ float64     │ 19    │                                    │
│ │ Number of columns │ 24     │ │ object      │ 4     │                                    │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓  │
│ ┃        ┃ missin ┃ comple ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p75   ┃ p100   ┃ hist  ┃  │
│ ┃        ┃ g      ┃ te     ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┃        ┃        ┃ rate   ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩  │
│ │ AllHom │      0 │      1 │  31000 │ 700000 │     0 │    730 │  4700 │ 350000 │   █   │  │
│ │ es_Tie │        │        │        │        │       │        │       │     00 │       │  │
│ │ r1     │        │        │        │        │       │        │       │        │       │  │
│ │ AllHom │      0 │      1 │  31000 │ 700000 │     0 │    770 │  4500 │ 350000 │   █   │  │
│ │ es_Tie │        │        │        │        │       │        │       │     00 │       │  │
│ │ r2     │        │        │        │        │       │        │       │        │       │  │
│ │ AllHom │      0 │      1 │  31000 │ 700000 │    14 │    780 │  4600 │ 350000 │   █   │  │
│ │ es_Tie │        │        │        │        │       │        │       │     00 │       │  │
│ │ r3     │        │        │        │        │       │        │       │        │       │  │
│ │ AllHom │      0 │      1 │  92000 │ 210000 │   110 │   2400 │ 13000 │ 110000 │   █   │  │
│ │ es_All │        │        │        │      0 │       │        │       │    000 │       │  │
│ │ Tiers  │        │        │        │        │       │        │       │        │       │  │
│ │ UWHome │      0 │      1 │   1300 │  14000 │     0 │     51 │   550 │ 590000 │   █   │  │
│ │ s_Tier │        │        │        │        │       │        │       │        │       │  │
│ │ 1      │        │        │        │        │       │        │       │        │       │  │
│ │ UWHome │      0 │      1 │   1200 │  13000 │     0 │     59 │   540 │ 540000 │   █   │  │
│ │ s_Tier │        │        │        │        │       │        │       │        │       │  │
│ │ 2      │        │        │        │        │       │        │       │        │       │  │
│ │ UWHome │      0 │      1 │   1600 │  16000 │     0 │    100 │   750 │ 730000 │   █   │  │
│ │ s_Tier │        │        │        │        │       │        │       │        │       │  │
│ │ 3      │        │        │        │        │       │        │       │        │       │  │
│ │ UWHome │      0 │      1 │   4100 │  43000 │   100 │    250 │  2000 │ 190000 │   █   │  │
│ │ s_AllT │        │        │        │        │       │        │       │      0 │       │  │
│ │ iers   │        │        │        │        │       │        │       │        │       │  │
│ │ UWHome │      0 │      1 │ 280000 │ 280000 │     0 │ 760000 │ 11000 │ 120000 │   █   │  │
│ │ s_Tota │        │        │    000 │   0000 │       │      0 │  0000 │ 000000 │       │  │
│ │ lValue │        │        │        │        │       │        │       │        │       │  │
│ │ _T     │        │        │        │        │    

## pandas profiling

If you want something more in-depth, try `pandas-profiling`. We can install `pandas-profiling` via Anaconda. To use this package, you'll need to type the following in the terminal (Mac) or command prompt (Windows). You'll need to restart VS Code.

```
conda install -c conda-forge pandas-profiling
```

You can [read all about it on their Github page](https://github.com/pandas-profiling/pandas-profiling). It's ridiculous what it can do. 

Let's add the `import` needed to bring `ProfileReport` from `pandas_profiling`.

In [ ]:
from pandas_profiling import ProfileReport
uw = pd.read_csv('https://github.com/aaiken1/fin-data-analysis-python/raw/main/data/zestimatesAndCutoffs_byGeo_uw_2017-10-10_forDataPage.csv')


This will create the report.

In [ ]:
profile = ProfileReport(uw, title="Zillow Housing Data for Flood Risk", minimal=True)

We can now view the report in a Jupyter widget. It will take a couple of minutes to summarize everything. 

In [ ]:
#profile.to_widgets()

You can also create an HTML file. Using the code below, the file will end up in the same folder as your `.ipynb` notebook. I've commented it out so that it doesn't run and create large file everytime.

In [ ]:
#profile.to_file("nc_profile.html")

Finally, I'll create a profile that will appear in this `.ipynb`.

In [ ]:
profile.to_notebook_iframe()